In [49]:
import os
import json
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
from gensim.corpora import Dictionary
from gensim.corpora.bleicorpus import BleiCorpus

In [2]:
# load args
with open("../args.json", "r") as f:
    args = json.load(f)

# Load Model Output

In [45]:
a.

<bound method Dictionary.__iter__ of <gensim.corpora.dictionary.Dictionary object at 0x7f79b20a9190>>

In [42]:
a = Dictionary.load(path_dict)

In [99]:
path_corpus = f'{args["corpus"]}{args["slice_type"]}'
dict_files = sorted([file for file in os.listdir(path_corpus) if ".dict" in file])
path_models = f'{args["results"]}hdp/{args["slice_type"]}'
models_dir = sorted(os.listdir(path_models))

slices = range(1, len(models_dir)+1)
data = {}
for slice in slices:
    # load dictionary {word->id}
    path_dict = f'{path_corpus}/{dict_files[slice-1]}'
    token2id = Dictionary.load(path_dict).token2id
    vocabulary = list(token2id.keys())
    
    # load topics distributions
    path_topics = f'{path_models}/{models_dir[slice-1]}/mode-topics.dat'
    with open(path_topics, "r") as f:
        topics = np.array([[int(word) for word in line.strip().split()] for line in f])
    topics_dists = (topics.T/topics.sum(axis=1)).T
    
    # load word assignments and get mixture weigths
    path_word_assignments = f"{path_models}/{models_dir[slice-1]}/mode-word-assignments.dat"
    word_assignments = []
    with open(path_word_assignments, "r") as f:
        lines = f.readlines()[1:]
        # (doc_id, word_id, topic_id, x)
        for line in lines:
            line = line.strip().split() 
            word_assignment = {"doc_id": int(line[0]) ,"word_id": int(line[1]), "topic_id": int(line[2])}
            word_assignments.append(word_assignment)
    
    # for each document get the number of words that each topic has
    word_assignments = pd.DataFrame(word_assignments, columns=["doc_id", "word_id", "topic_id"])
    term_frequency = word_assignments.groupby("word_id").size().values
    word_assignments = word_assignments.pivot_table(index="doc_id", columns="topic_id", aggfunc="count")
    word_assignments.fillna(0, inplace=True)
    
    # for each document get the number of words
    doc_length = word_assignments.sum(axis=1) 
    
    # for each document get the topic proportions that draw the document (\pi_d)
    doc_topic_dists = word_assignments.divide(doc_length, axis=0).values

    #save data in a dict
    data[slice] = {"topic_term_dists": topics_dists, "doc_topic_dists": doc_topic_dists, 
                   "term_frequency": term_frequency, "doc_lengths": doc_length.values, "vocab": vocabulary}

# Visualizations

In [102]:
hdp_ldavis_data = pyLDAvis.prepare(mds="tsne", **data[1])

In [103]:
pyLDAvis.display(hdp_ldavis_data)